## Set up

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

load_dotenv()

doublecheck_env("../.env")

OLLAMA_MODEL=****a3.2
OPENAI_API_KEY=****upAA
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=****.com
LANGSMITH_API_KEY=****7b12
LANGSMITH_PROJECT=****ject


## Models and Messages

Models docs: https://docs.langchain.com/oss/python/integrations/chat

Messages types:
- SystemMessage
- HumanMessage
- AIMessage
- ToolMessage

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

In [3]:
@tool
def get_stock_price(symbol: str) -> str:
    """Get today's stock price for a given symbol."""
    # dummy logic and price
    prices = {
        "BBCA": 9500,
        "BMRI": 7200,
        "TLKM": 3100,
    }
    price = prices.get(symbol.upper(), None)

    if price is None:
        return f"Symbol {symbol} not found."

    return f"The current price of {symbol} is {price}."

In [4]:
SYSTEM_PROMPT = "You are a holistic stock market analyst in Indonesia, you will provide data the user ask, you can use tools to get the data"


agent = create_agent(
    model="gpt-5-mini",
    tools=[get_stock_price],
    system_prompt=SYSTEM_PROMPT
)

In [5]:
human_msg = HumanMessage(content="Give price for BBCA and BMRI.")
result = agent.invoke({"messages": [human_msg]})

In [6]:
print(result["messages"][-1].content)

Current prices (IDR per share, on IDX):
- BBCA: Rp 9.500
- BMRI: Rp 7.200

Would you like intraday change, percentage move, volume, or a short trade/long view for either stock?


In [7]:
result["messages"]

[HumanMessage(content='Give price for BBCA and BMRI.', additional_kwargs={}, response_metadata={}, id='f7c093e5-f545-4c52-9058-d02d73de4fca'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 167, 'total_tokens': 288, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CjQiGSROgRp52MPACrEWAl31XpPPt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeed4-4112-7950-b085-5a0dbd4a6917-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'BBCA'}, 'id': 'call_pajcLQeU1TZnfZxCJXDZ3iwr', 'type': 'tool_call'}, {'name': 'get_stock_price', 'args': {'symbol': 'BMRI'}, 'id': 'call_GnqpDqtG3lzqQBE8

In [8]:
for i in range(len(result["messages"])):
    print(type(result["messages"][i]))


<class 'langchain_core.messages.human.HumanMessage'>
<class 'langchain_core.messages.ai.AIMessage'>
<class 'langchain_core.messages.tool.ToolMessage'>
<class 'langchain_core.messages.tool.ToolMessage'>
<class 'langchain_core.messages.ai.AIMessage'>


In [10]:
for msg in result["messages"]:
    print(f"{msg.type.capitalize()}: {msg.content}\n")

Human: Give price for BBCA and BMRI.

Ai: 

Tool: The current price of BBCA is 9500.

Tool: The current price of BMRI is 7200.

Ai: Current prices (IDR per share, on IDX):
- BBCA: Rp 9.500
- BMRI: Rp 7.200

Would you like intraday change, percentage move, volume, or a short trade/long view for either stock?



In [11]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Give price for BBCA and BMRI.
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_pajcLQeU1TZnfZxCJXDZ3iwr)
 Call ID: call_pajcLQeU1TZnfZxCJXDZ3iwr
  Args:
    symbol: BBCA
  get_stock_price (call_GnqpDqtG3lzqQBE89JV94WIv)
 Call ID: call_GnqpDqtG3lzqQBE89JV94WIv
  Args:
    symbol: BMRI
================================= Tool Message =================================
Name: get_stock_price

The current price of BBCA is 9500.
================================= Tool Message =================================
Name: get_stock_price

The current price of BMRI is 7200.
================================== Ai Message ==================================

Current prices (IDR per share, on IDX):
- BBCA: Rp 9.500
- BMRI: Rp 7.200

Would you like intraday change, percentage move, volume, or a short trade/long view for either stock?


## Streaming

In [12]:
for step in agent.stream(
    {"messages": [human_msg]},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Give price for BBCA and BMRI.
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_3INBGZwZCMmua8P6bTbTRv7Z)
 Call ID: call_3INBGZwZCMmua8P6bTbTRv7Z
  Args:
    symbol: BBCA
  get_stock_price (call_zMX6prJDsgtMfliijmeUMEb1)
 Call ID: call_zMX6prJDsgtMfliijmeUMEb1
  Args:
    symbol: BMRI
================================= Tool Message =================================
Name: get_stock_price

The current price of BMRI is 7200.
================================== Ai Message ==================================

Harga saat ini:
- BBCA: Rp 9.500 per saham
- BMRI: Rp 7.200 per saham

Harga ini merupakan kutipan saat ini dan dapat berubah cepat. Mau saya tampilkan perubahan harian (%) atau grafik harga?


In [13]:
for token, metadata in agent.stream(
    {"messages": [human_msg]},
    stream_mode="messages"
):
    print(f"{token.content}", end="")

Symbol BBCA.JK not found.Symbol BMRI.JK not found.The current price of BBCA is 9500.The current price of BMRI is 7200.Current prices on IDX (as of 2025-12-05):

- BBCA (Bank Central Asia) — Rp 9.500 per share  
- BMRI (Bank Mandiri) — Rp 7.200 per share

Want intraday change, a price chart, recent news, or valuation/technical notes for either stock?

In [ ]:
for step in agent.stream(
    {"messages": [human_msg]},
    stream_mode=["custom"] # ["custom", "values", "messages"]
):
    print(step)